# 금융정보
> * 코스피 지수
> * 코스닥 지수
> * 달러/원화 환율  

기간 : 2017년 12월 01일 ~

In [2]:
def Get_Finance_Info(target, symbol, date, nowdate):
    from pandas_datareader import DataReader
    print (target, symbol, date, nowdate)
    value = True
    while value:
        try:
            df = DataReader(symbol, "yahoo", date, nowdate)
            print (df.shape)
        except:
            value = True
        else:
            value = False
    return df.add_suffix('_'+target)

def Get_Finance(date): 
    import pandas as pd
    import chat_bot as cb
    from datetime import datetime
    nowdate = datetime.now().date()
    date = datetime.strptime(str(date),'%Y%m%d').date()
    target = {'KOSPI' : '^KS11',
             'KOSDAQ' : '^KQ11',
             'CURRENCY' : 'KRW=X',
             'NASDAQ' : '^IXIC',
             'DOW_JONES' : '^DJI',
             'S&P500' : '^GSPC',
             'GOLD_FUTURES' : 'GC=F',
             'BITCOIN' : 'BTC-USD',
             'NIKKEI_225' : '^N225',
             'HANG_SENG_INDEX' : '^HSI'}
    df = list(map(lambda x: Get_Finance_Info(x, target[x], date, nowdate), target))
    df = pd.concat(df, axis = 1)
    df = df.dropna()
    df.reset_index(inplace = True)
    df['Date'] = df.Date.dt.strftime('%Y.%m.%d')
    #slack = cb.Slacker(cb.slacktoken())
    #strNowDate = nowdate.strftime('%Y-%m-%d')
    #slack.chat.post_message('# notification',
     #                       '''targetData : {}, {}, {}
      #                      from : {}
       #                     to : {} '''.format(*target, date, strNowDate))
    return df

from datetime import datetime
import Database_Handler as dh
startTime = datetime.now()
dateIs = startTime.date().strftime('%Y%m%d')
dateIs = '20171201'
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
df_finance = Get_Finance(dateIs)
df_finance.drop_duplicates(subset='Date', inplace = True)
#financeCollectionName = 'Finance'
#useCollection_Finance = dh.Use_Collection(useDb, financeCollectionName)
#dateIsInDB = useCollection_Finance.distinct('Date')
#df_finance = df_finance[~df_finance.Date.isin(dateIsInDB)]
#useCollection_Finance.insert_many(df_finance.to_dict('records'))
endTime = datetime.now()
runningTime= endTime - startTime
print (runningTime)

KOSPI ^KS11 2017-12-01 2018-02-28
(59, 6)
KOSDAQ ^KQ11 2017-12-01 2018-02-28
(59, 6)
CURRENCY KRW=X 2017-12-01 2018-02-28
(65, 6)
NASDAQ ^IXIC 2017-12-01 2018-02-28
(60, 6)
DOW_JONES ^DJI 2017-12-01 2018-02-28
(60, 6)
S&P500 ^GSPC 2017-12-01 2018-02-28
(60, 6)
GOLD_FUTURES GC=F 2017-12-01 2018-02-28
(74, 6)
BITCOIN BTC-USD 2017-12-01 2018-02-28
(91, 6)
NIKKEI_225 ^N225 2017-12-01 2018-02-28
(65, 6)
HANG_SENG_INDEX ^HSI 2017-12-01 2018-02-28
(60, 6)
0:00:15.889952


In [3]:
df_finance

,Date,Open_KOSPI,High_KOSPI,Low_KOSPI,Close_KOSPI,Adj Close_KOSPI,Volume_KOSPI,Open_KOSDAQ,High_KOSDAQ,Low_KOSDAQ,...,Low_NIKKEI_225,Close_NIKKEI_225,Adj Close_NIKKEI_225,Volume_NIKKEI_225,Open_HANG_SENG_INDEX,High_HANG_SENG_INDEX,Low_HANG_SENG_INDEX,Close_HANG_SENG_INDEX,Adj Close_HANG_SENG_INDEX,Volume_HANG_SENG_INDEX
0,2017.12.01,2488.919922,2488.919922,2472.750000,2475.409912,2475.409912,288600.0,779.539978,788.150024,776.710022,...,22675.070313,22819.029297,22819.029297,89700.0,29261.310547,29345.970703,29028.759766,29074.240234,29074.240234,2.086681e+09
1,2017.12.04,2486.719971,2501.669922,2476.290039,2501.669922,2501.669922,323800.0,788.020020,788.460022,778.200012,...,22693.990234,22707.160156,22707.160156,68900.0,28951.189453,29343.779297,28803.189453,29138.279297,29138.279297,1.624072e+09
2,2017.12.05,2490.729980,2513.679932,2487.149902,2510.120117,2510.120117,395400.0,782.140015,783.609985,773.799988,...,22522.279297,22622.380859,22622.380859,75900.0,28898.580078,29124.240234,28838.119141,28842.800781,28842.800781,1.439200e+09
3,2017.12.06,2510.189941,2510.469971,2474.290039,2474.370117,2474.370117,507600.0,778.280029,783.539978,765.250000,...,22119.210938,22177.039063,22177.039063,97300.0,28850.359375,28929.210938,28204.529297,28224.800781,28224.800781,2.605434e+09
4,2017.12.07,2479.550049,2480.850098,2452.399902,2461.979980,2461.979980,404900.0,771.469971,773.280029,744.340027,...,22317.150391,22498.029297,22498.029297,79500.0,28366.140625,28478.339844,28134.929688,28303.189453,28303.189453,1.782454e+09
5,2017.12.08,2470.090088,2473.169922,2459.020020,2464.000000,2464.000000,406400.0,757.210022,760.729980,743.900024,...,22625.410156,22811.080078,22811.080078,106900.0,28400.009766,28705.759766,28344.390625,28639.849609,28639.849609,1.747377e+09
6,2017.12.11,2467.810059,2471.989990,2457.979980,2471.489990,2471.489990,240100.0,747.989990,764.090027,745.880005,...,22787.400391,22938.730469,22938.730469,69500.0,28645.410156,28998.509766,28645.410156,28965.289063,28965.289063,2.127756e+09
7,2017.12.12,2471.229980,2471.429932,2455.389893,2461.000000,2461.000000,281200.0,768.299988,769.140015,755.030029,...,22834.890625,22866.169922,22866.169922,86100.0,28972.740234,29066.769531,28745.730469,28793.880859,28793.880859,2.135704e+09
8,2017.12.13,2462.419922,2481.919922,2457.699951,2480.550049,2480.550049,255000.0,764.330017,773.020020,763.169983,...,22697.320313,22758.070313,22758.070313,91400.0,28856.779297,29276.669922,28707.599609,29222.099609,29222.099609,2.574054e+09
9,2017.12.14,2486.570068,2514.610107,2469.479980,2469.479980,2469.479980,317800.0,777.250000,777.440002,764.739990,...,22638.759766,22694.449219,22694.449219,85100.0,29335.759766,29367.509766,29047.609375,29166.380859,29166.380859,1.959306e+09
